# 📌 Objectif  : Régression (Supervisé) :
Employabilité d’un candidat : Prédire le délai d’employabilité (en jours) après le diplôme




## Connexion pour ssms

In [13]:
pip install sqlalchemy pymssql



   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.5/2.0 MB 1.1 MB/s eta 0:00:02
   ---------- ----------------------------- 0.5/2.0 MB 1.1 MB/s eta 0:00:02
   --------------- ------------------------ 0.8/2.0 MB 1.0 MB/s eta 0:00:02
   -------------------- ------------------- 1.0/2.0 MB 1.1 MB/s eta 0:00:01
   -------------------- ------------------- 1.0/2.0 MB 1.1 MB/s eta 0:00:01
   -------------------- ------------------- 1.0/2.0 MB 1.1 MB/s eta 0:00:01
   ------------------------- -------------- 1.3/2.0 MB 789.0 kB/s eta 0:00:01
   ------------------------- -------------- 1.3/2.0 MB 789.0 kB/s eta 0:00:01
   ------------------------------ --------- 1.6/2.0 MB 676.3 kB/s eta 0:00:01
   ------------------------------ --------- 1.6/2.0 MB 676.3 kB/s eta 0:00:01
   ---------------------

In [4]:
import pandas as pd
from sqlalchemy import create_engine

# Paramètres de connexion
server = 'LAPTOP-VAILS20L'
database = 'DW_PI'
username = 'sa'
password = 'sa123'
driver = 'ODBC Driver 17 for SQL Server'  # Utiliser le driver correct

# Créer la chaîne de connexion pour SQLAlchemy
connection_string = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver={driver}"

# Créer l'engin SQLAlchemy
engine = create_engine(connection_string)

# Requête SQL
query = """
SELECT 
    e.Etudiant_Pk, 
    e.diplome, 
    e.dateDiplome, 
    e.date_firstjob, 
    f.Fk_Entreprise, 
    f.Fk_Offre, 
    f.FK_date1, 
    f.FK_date2,
    en.name AS entreprise_name,
    o.offre AS offre_name
FROM 
    [DW_PI].[dbo].[Dim_Etudiant] e
JOIN 
    [DW_PI].[dbo].[Fact_employ1] f
ON 
    e.Etudiant_Pk = f.Etudiant_Fk
JOIN 
    [DW_PI].[dbo].[Dim_Entreprise] en
ON 
    f.Fk_Entreprise = en.Pk_Entreprise
JOIN 
    [DW_PI].[dbo].[Dim_Offre] o
ON 
    f.Fk_Offre = o.Pk_Offre
"""

# Charger les données dans un DataFrame
df = pd.read_sql(query, engine)

# Afficher les premières lignes des données pour vérifier
print(df.head())


   Etudiant_Pk                                            diplome dateDiplome  \
0          280  Ecole Supérieure Privée d'Ingénierie et de Tec...  2013-07-13   
1          387  Université libre de Tunis, Software engineerin...  2013-11-06   
2          387  Université libre de Tunis, Software engineerin...  2013-11-06   
3          398  Ecole Supérieure Privée d'Ingénierie et de Tec...  2017-09-05   
4          398  Ecole Supérieure Privée d'Ingénierie et de Tec...  2017-09-05   

  date_firstjob  Fk_Entreprise  Fk_Offre  FK_date1  FK_date2  \
0    2013-09-29             22       223  20130713  20130929   
1    2015-09-09             11        17  20131106  20150909   
2    2015-09-09             11       141  20131106  20150909   
3    2019-11-22             23        27  20170905  20191122   
4    2019-11-22              1        47  20170905  20191122   

                              entreprise_name                      offre_name  
0                                       Value   

In [4]:
print(df.columns)


Index(['Etudiant_Pk', 'diplome', 'dateDiplome', 'date_firstjob',
       'Fk_Entreprise', 'Fk_Offre', 'FK_date1', 'FK_date2', 'entreprise_name',
       'offre_name'],
      dtype='object')


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Création de la cible : nombre de jours entre diplôme et premier emploi
df['dateDiplome'] = pd.to_datetime(df['dateDiplome'])
df['date_firstjob'] = pd.to_datetime(df['date_firstjob'])
df['jours_attente'] = (df['date_firstjob'] - df['dateDiplome']).dt.days

# Suppression des lignes avec valeurs manquantes dans la cible
df = df.dropna(subset=['jours_attente'])

# Sélection des features
features = df[['entreprise_name', 'offre_name', 'dateDiplome']]
features['annee_diplome'] = features['dateDiplome'].dt.year
features['mois_diplome'] = features['dateDiplome'].dt.month
X = features[['entreprise_name', 'offre_name', 'annee_diplome', 'mois_diplome']]
y = df['jours_attente']

# Encodage des variables catégorielles
X = pd.get_dummies(X, columns=['entreprise_name', 'offre_name'], drop_first=True)

# Split Train/Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modèles
lr = LinearRegression()
rf = RandomForestRegressor(random_state=42)

lr.fit(X_train, y_train)
rf.fit(X_train, y_train)


C:\Users\souha\AppData\Local\Temp\ipykernel_26864\1804541414.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['annee_diplome'] = features['dateDiplome'].dt.year
C:\Users\souha\AppData\Local\Temp\ipykernel_26864\1804541414.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['mois_diplome'] = features['dateDiplome'].dt.month


RandomForestRegressor(random_state=42)

In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Imaginons que df soit ton DataFrame déjà préparé

# Création de la cible : nombre de jours entre diplôme et premier emploi
df['dateDiplome'] = pd.to_datetime(df['dateDiplome'])
df['date_firstjob'] = pd.to_datetime(df['date_firstjob'])
df['jours_attente'] = (df['date_firstjob'] - df['dateDiplome']).dt.days

# Suppression des lignes avec valeurs manquantes dans la cible
df = df.dropna(subset=['jours_attente'])

# Sélection des features
features = df[['entreprise_name', 'offre_name', 'dateDiplome']]
features['annee_diplome'] = features['dateDiplome'].dt.year
features['mois_diplome'] = features['dateDiplome'].dt.month
X = features[['entreprise_name', 'offre_name', 'annee_diplome', 'mois_diplome']]
y = df['jours_attente']

# Encodage des variables catégorielles
X = pd.get_dummies(X, columns=['entreprise_name', 'offre_name'], drop_first=True)

# Split Train/Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialisation du modèle RandomForestRegressor
rf = RandomForestRegressor(random_state=42)

# Entraînement du modèle
rf.fit(X_train, y_train)

# Prédictions
y_pred = rf.predict(X_test)

# Évaluation du modèle
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"MSE : {mse:.2f}")
print(f"RMSE : {rmse:.2f}")
print(f"R² : {r2:.2f}")


C:\Users\souha\AppData\Local\Temp\ipykernel_7064\661835527.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['annee_diplome'] = features['dateDiplome'].dt.year
C:\Users\souha\AppData\Local\Temp\ipykernel_7064\661835527.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['mois_diplome'] = features['dateDiplome'].dt.month


MSE : 78648.72
RMSE : 280.44
R² : 0.36


## Modele 2 catboost

In [2]:
!pip install catboost


   ---------------------------------------- 0.0/102.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/102.5 MB ? eta -:--:--
   ---------------------------------------- 0.3/102.5 MB ? eta -:--:--
   ---------------------------------------- 0.5/102.5 MB 1.1 MB/s eta 0:01:32
   ---------------------------------------- 0.8/102.5 MB 1.1 MB/s eta 0:01:31
   ---------------------------------------- 0.8/102.5 MB 1.1 MB/s eta 0:01:31
   ---------------------------------------- 1.0/102.5 MB 1.0 MB/s eta 0:01:39
    --------------------------------------- 1.3/102.5 MB 958.5 kB/s eta 0:01:46
    --------------------------------------- 1.6/102.5 MB 1.0 MB/s eta 0:01:40
    --------------------------------------- 1.6/102.5 MB 1.0 MB/s eta 0:01:40
    --------------------------------------- 1.8/102.5 MB 1.0 MB/s eta 0:01:39
    --------------------------------------- 2.1/102.5 MB 1.0 MB/s eta 0:01:41
    --------------------------------------- 2.4/102.5 MB 1.0 MB/s eta 0:01:37
   - 

In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_validate
from sklearn.metrics import mean_squared_error, r2_score
from catboost import CatBoostRegressor
from sklearn.base import BaseEstimator, RegressorMixin

# 1. Dates & création de la cible
df['dateDiplome'] = pd.to_datetime(df['dateDiplome'])
df['date_firstjob'] = pd.to_datetime(df['date_firstjob'])
df['jours_attente'] = (df['date_firstjob'] - df['dateDiplome']).dt.days

# 2. Nettoyage
df = df[df['jours_attente'] >= 0]
df = df.dropna(subset=['jours_attente'])

# 3. Feature bonus
df['annee_diplome'] = df['dateDiplome'].dt.year
df['mois_diplome'] = df['dateDiplome'].dt.month
df['jours_depuis_diplome'] = (pd.to_datetime('today') - df['dateDiplome']).dt.days
df['jour_semaine_diplome'] = df['dateDiplome'].dt.weekday
df['diff_annees_diplome'] = (pd.to_datetime('today') - df['dateDiplome']).dt.days / 365

# 4. Variables explicatives et cible
df['entreprise_name'] = df['entreprise_name'].astype('category')
df['offre_name'] = df['offre_name'].astype('category')

X = df[['entreprise_name', 'offre_name', 'annee_diplome', 'mois_diplome', 
        'jours_depuis_diplome', 'jour_semaine_diplome', 'diff_annees_diplome']]
y = df['jours_attente']

# 5. Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 6. Indices des colonnes catégorielles (dans X)
cat_features = [0, 1]  # positions dans X

# 7. Paramètres pour la recherche
param_dist = {
    'depth': [6, 8, 10, 12],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'iterations': [300, 500, 1000],
    'l2_leaf_reg': [1, 3, 5, 7]
}

# 8. Modèle de base
catboost_model = CatBoostRegressor(random_state=42, verbose=0)

# 9. Recherche d'hyperparamètres
random_search = RandomizedSearchCV(estimator=catboost_model, param_distributions=param_dist,
                                   n_iter=10, cv=3, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train, cat_features=cat_features)

# 10. Résultats
print("Meilleurs paramètres trouvés:", random_search.best_params_)

# 11. Meilleur modèle
best_model = random_search.best_estimator_

# 12. Prédiction
y_pred_best = best_model.predict(X_test)

# 13. Évaluation
mse_best = mean_squared_error(y_test, y_pred_best)
rmse_best = np.sqrt(mse_best)
r2_best = r2_score(y_test, y_pred_best)

print(f"CatBoost Optimisé MSE : {mse_best:.2f}")
print(f"CatBoost Optimisé RMSE : {rmse_best:.2f}")
print(f"CatBoost Optimisé R² : {r2_best:.2f}")




Meilleurs paramètres trouvés: {'learning_rate': 0.05, 'l2_leaf_reg': 7, 'iterations': 1000, 'depth': 6}
CatBoost Optimisé MSE : 33426.15
CatBoost Optimisé RMSE : 182.83
CatBoost Optimisé R² : 0.70


## Test

In [22]:
# Exemple d'observation pour prédiction
new_data = pd.DataFrame({
    'entreprise_name': ['Entreprise A'],
    'offre_name': ['Offre B'],
    'dateDiplome': [pd.to_datetime('2024-06-15')]
})

# Ajout des mêmes features que dans ton dataset d'entraînement
new_data['annee_diplome'] = new_data['dateDiplome'].dt.year
new_data['mois_diplome'] = new_data['dateDiplome'].dt.month
new_data['jours_depuis_diplome'] = (pd.to_datetime('today') - new_data['dateDiplome']).dt.days
new_data['jour_semaine_diplome'] = new_data['dateDiplome'].dt.weekday
new_data['diff_annees_diplome'] = new_data['jours_depuis_diplome'] / 365

# Sélectionner les bonnes colonnes (dans le bon ordre)
X_new = new_data[['entreprise_name', 'offre_name', 'annee_diplome', 'mois_diplome', 
                  'jours_depuis_diplome', 'jour_semaine_diplome', 'diff_annees_diplome']]

# Assurer le bon type pour les variables catégorielles
X_new['entreprise_name'] = X_new['entreprise_name'].astype('category')
X_new['offre_name'] = X_new['offre_name'].astype('category')

# Prédiction
pred = best_model.predict(X_new)

print(f"Nombre estimé de jours entre diplôme et emploi : {pred[0]:.2f} jours")


Nombre estimé de jours entre diplôme et emploi : 824.29 jours


C:\Users\souha\AppData\Local\Temp\ipykernel_26864\1895403630.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_new['entreprise_name'] = X_new['entreprise_name'].astype('category')
C:\Users\souha\AppData\Local\Temp\ipykernel_26864\1895403630.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_new['offre_name'] = X_new['offre_name'].astype('category')


## Deploiement de catBoost

In [23]:
import joblib

# Sauvegarde du modèle CatBoost
joblib.dump(best_model, "modele_catboost.pkl")

# Sauvegarde des colonnes utilisées
joblib.dump(X.columns.tolist(), "colonnes_model_catboost.pkl")


['colonnes_model_catboost.pkl']